# Capítulo 09: Arrow Compute Functions

## Tópicos Abordados:
- Operações vetorizadas
- String operations
- Filtros e máscaras
- Agregações
- Joins


In [ ]:
# Setup e Importações
import pyarrow as pa
import duckdb
import pandas as pd
import numpy as np

# Configuração de display do pandas
pd.set_option('display.max_rows', 10)

print('Bibliotecas importadas com sucesso!')

In [ ]:
# Criação de dados de exemplo para o capítulo
data = pa.table({
    'id': range(10000),
    'valor': np.random.randn(10000),
    'categoria': np.random.choice(['A', 'B', 'C'], 10000)
})

print(f'Dataset criado com {len(data)} registros')
print(data.schema)

## Operações vetorizadas
Exemplos e exercícios sobre operações vetorizadas.

In [ ]:
# Prática: Operações vetorizadas
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## String operations
Exemplos e exercícios sobre string operations.

In [ ]:
# Prática: String operations
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Filtros e máscaras
Exemplos e exercícios sobre filtros e máscaras.

In [ ]:
# Prática: Filtros e máscaras
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Agregações
Exemplos e exercícios sobre agregações.

In [ ]:
# Prática: Agregações
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()

## Joins
Exemplos e exercícios sobre joins.

In [ ]:
# Prática: Joins
con = duckdb.connect()
# Implemente seu código aqui

# Exemplo básico:
# con.execute('SELECT * FROM data LIMIT 5').arrow()